In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from operator import itemgetter
%matplotlib inline

In [3]:
results = {'eval': {'auc': [0.499306, 0.494721, 0.488865, 0.478342, 0.5674]},
 'train': {'auc': [0.557143, 0.632473, 0.715275, 0.74478, 0.780549]}}

labels = ['max_depth','eta','num_rounds','eval_reslts']
data = [[3,0.1,100,results],[3,0.1,100,results],[3,0.1,100,results]]
df = pd.DataFrame(data=data,columns=labels)

In [4]:
df

,max_depth,eta,num_rounds,eval_reslts
0,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715..."
1,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715..."
2,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715..."


In [2]:
def myfunc(x):
    
    val = np.array(x['eval']['auc'])
    maxInd = np.argmax(val)
    maxVal = val[maxInd]
    
    return (maxInd,maxVal)
    

In [21]:
test = df['eval_reslts'].loc[0]
print test['eval'].values

<built-in method values of dict object at 0x7fc0bbb6d168>


In [22]:
test = df['eval_reslts'].loc[0]
myfunc(test)

(4, 0.56740000000000002)

In [24]:
df['eval_auc'] = df['eval_reslts'].map(lambda x: myfunc(x))

In [25]:
df

,max_depth,eta,num_rounds,eval_reslts,eval_auc
0,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715...","(4, 0.5674)"
1,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715...","(4, 0.5674)"
2,3,0.1,100,"{u'train': {u'auc': [0.557143, 0.632473, 0.715...","(4, 0.5674)"


In [2]:
def build_roc(df):
    
    df['TPR'] = df['recall']
    df['FPR'] = df['FP']/(df['FP'] + df['TN'])
    
#     plt.plot([0,1],[0,1],'k',linewidth=0.5)
    plt.figure()
    plt.plot(df.FPR.values,df.TPR.values,'r*',markersize=7)
    plt.xlabel('FPR')
    plt.xlim([0,1])
    plt.ylabel('TPR')
    plt.ylim([0,1])
    titlestr = "AUC: {}  k = {}".format(np.trapz(df.TPR.values[::-1],x=df.FPR.values[::-1]),int(df.k.unique()))
    plt.title(titlestr)

In [ ]:
for k in df.k.unique():
    build_roc(df[df['k']==k])

In [3]:
path = '../../data/gridsearch_xgb.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
df.head()

,Unnamed: 0,num_rounds,max_depth,eta,eval_results
0,0,100,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0...."
1,1,300,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0...."
2,2,600,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0...."
3,3,900,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0...."
4,4,100,4,0.03,"{'train': {'auc': ['0.564531', '0.564540', '0...."


In [23]:
df['eval_results'].ix[0]

"{'train': {'auc': ['0.552844', '0.552849', '0.552853', '0.552854', '0.564506', '0.564508', '0.564585', '0.570304', '0.570369', '0.570379', '0.574745', '0.574746', '0.574748', '0.574748', '0.574755', '0.574814', '0.577942', '0.577889', '0.577944', '0.591977', '0.591976', '0.591979', '0.595014', '0.605616', '0.609023', '0.609023', '0.609026', '0.609065', '0.615975', '0.616123', '0.616305', '0.616303', '0.616452', '0.616523', '0.616298', '0.616303', '0.648427', '0.648950', '0.648568', '0.652931', '0.652968', '0.652983', '0.653288', '0.653320', '0.657873', '0.657791', '0.657966', '0.657924', '0.658021', '0.658081', '0.658085', '0.658091', '0.663862', '0.663871', '0.663874', '0.663901', '0.665859', '0.668477', '0.668617', '0.668639', '0.668658', '0.668621', '0.668623', '0.670125', '0.670147', '0.670208', '0.669043', '0.671257', '0.671260', '0.677831', '0.680031', '0.680049', '0.680268', '0.680302', '0.680307', '0.680937', '0.680934', '0.682168', '0.682185', '0.682234', '0.682249', '0.68382

In [6]:
def findtrain(x):
    badset = {"{","}","'",",","t","r","a","i","n","u","c",":","[","]"}
    outstr = ""
    results = x.split("eval")
    
    for char in results[0]:
        if char not in badset:
            outstr = outstr + char
            
    numbers = outstr.split(" ")
    aucVals = np.array([float(number) for number in numbers[2:-1]])
    maxInd = np.argmax(aucVals)
    maxVal = aucVals[maxInd]
    
    return (maxInd,maxVal)
  
def findbest(x):
    numvals = len(x)
    bestrow = None
    bestVal = None
    
    for row in xrange(numvals):
        if x[row][1] > bestVal:
            bestrow = row
            bestVal = x[row][1]
            
    return bestrow, x[bestrow]


def findeval(x):
    badset = {"{","}","'",",","t","r","a","i","n","u","c",":","[","]"}
    outstr = ""
    results = x.split("eval")
    
    for char in results[1]:
        if char not in badset:
            outstr = outstr + char
            
    numbers = outstr.split(" ")
    aucVals = np.array([float(number) for number in numbers[2:]])
    maxInd = np.argmax(aucVals)
    maxVal = aucVals[maxInd]
    
    return (maxInd,maxVal)
    

In [7]:
df['eval_auc'] = df['eval_results'].map(lambda x: findeval(x))

In [8]:
df['train_auc'] = df['eval_results'].map(lambda x: findtrain(x))

In [9]:
df

,Unnamed: 0,num_rounds,max_depth,eta,eval_results,eval_auc,train_auc
0,0,100,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0....","(91, 0.7043)","(99, 0.697191)"
1,1,300,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0....","(284, 0.74588)","(299, 0.743419)"
2,2,600,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0....","(596, 0.80778)","(599, 0.767899)"
3,3,900,3,0.03,"{'train': {'auc': ['0.552844', '0.552849', '0....","(661, 0.80968)","(897, 0.782626)"
4,4,100,4,0.03,"{'train': {'auc': ['0.564531', '0.564540', '0....","(97, 0.70845)","(98, 0.708373)"
5,5,300,4,0.03,"{'train': {'auc': ['0.564531', '0.564540', '0....","(292, 0.77796)","(299, 0.753988)"
6,6,600,4,0.03,"{'train': {'auc': ['0.564531', '0.564540', '0....","(522, 0.81082)","(599, 0.77964)"
7,7,900,4,0.03,"{'train': {'auc': ['0.564531', '0.564540', '0....","(899, 0.81823)","(899, 0.792982)"
8,8,100,5,0.03,"{'train': {'auc': ['0.570415', '0.570409', '0....","(91, 0.71407)","(99, 0.718837)"
9,9,300,5,0.03,"{'train': {'auc': ['0.570415', '0.570409', '0....","(280, 0.79814)","(299, 0.76341)"


In [63]:
max(df['eval_auc'].values,key=itemgetter(1)) 

(163, 0.86024999999999996)

In [67]:
findbest(df['eval_auc'].values)

(53, (163, 0.86024999999999996))

In [68]:
df.ix[53]

Unnamed: 0                                                     53
num_rounds                                                    300
max_depth                                                       4
eta                                                           0.3
eval_results    {'train': {'auc': ['0.564531', '0.574791', '0....
eval_auc                                           (163, 0.86025)
train_auc                                         (299, 0.829813)
Name: 53, dtype: object

In [34]:
results = x.split("eval")

In [51]:
results[0]

"{'train': {'auc': ['0.552844', '0.552849', '0.552853', '0.552854', '0.564506', '0.564508', '0.564585', '0.570304', '0.570369', '0.570379', '0.574745', '0.574746', '0.574748', '0.574748', '0.574755', '0.574814', '0.577942', '0.577889', '0.577944', '0.591977', '0.591976', '0.591979', '0.595014', '0.605616', '0.609023', '0.609023', '0.609026', '0.609065', '0.615975', '0.616123', '0.616305', '0.616303', '0.616452', '0.616523', '0.616298', '0.616303', '0.648427', '0.648950', '0.648568', '0.652931', '0.652968', '0.652983', '0.653288', '0.653320', '0.657873', '0.657791', '0.657966', '0.657924', '0.658021', '0.658081', '0.658085', '0.658091', '0.663862', '0.663871', '0.663874', '0.663901', '0.665859', '0.668477', '0.668617', '0.668639', '0.668658', '0.668621', '0.668623', '0.670125', '0.670147', '0.670208', '0.669043', '0.671257', '0.671260', '0.677831', '0.680031', '0.680049', '0.680268', '0.680302', '0.680307', '0.680937', '0.680934', '0.682168', '0.682185', '0.682234', '0.682249', '0.68382

In [54]:
outstr = ""
badset = {"{","}","'",",","t","r","a","i","n","u","c",":","[","]"}
for char in results[0]:
        if char not in badset:
            outstr = outstr + char

In [55]:
outstr.split(" ")

['',
 '',
 '0.552844',
 '0.552849',
 '0.552853',
 '0.552854',
 '0.564506',
 '0.564508',
 '0.564585',
 '0.570304',
 '0.570369',
 '0.570379',
 '0.574745',
 '0.574746',
 '0.574748',
 '0.574748',
 '0.574755',
 '0.574814',
 '0.577942',
 '0.577889',
 '0.577944',
 '0.591977',
 '0.591976',
 '0.591979',
 '0.595014',
 '0.605616',
 '0.609023',
 '0.609023',
 '0.609026',
 '0.609065',
 '0.615975',
 '0.616123',
 '0.616305',
 '0.616303',
 '0.616452',
 '0.616523',
 '0.616298',
 '0.616303',
 '0.648427',
 '0.648950',
 '0.648568',
 '0.652931',
 '0.652968',
 '0.652983',
 '0.653288',
 '0.653320',
 '0.657873',
 '0.657791',
 '0.657966',
 '0.657924',
 '0.658021',
 '0.658081',
 '0.658085',
 '0.658091',
 '0.663862',
 '0.663871',
 '0.663874',
 '0.663901',
 '0.665859',
 '0.668477',
 '0.668617',
 '0.668639',
 '0.668658',
 '0.668621',
 '0.668623',
 '0.670125',
 '0.670147',
 '0.670208',
 '0.669043',
 '0.671257',
 '0.671260',
 '0.677831',
 '0.680031',
 '0.680049',
 '0.680268',
 '0.680302',
 '0.680307',
 '0.680937',
 '

In [10]:
s = "ljklj{}".format(5)

In [11]:
s

'ljklj5'